In [1]:
import os
import sys
import tarfile
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from six.moves import urllib

sess = tf.Session()


In [2]:
#define hyper paramaters
bacth_size=128 
output_every=50
generations=20000
eval_every=500 #每训练500次，衡量一次模型
image_width=32
image_height=32

crop_height=24 #size of cropped images will take
crop_width=24

num_channels=3
num_targets=10 #十分类

  


In [3]:
#在model的迭代过程中，适当降低lr（指数减少）
#newLr=0.1 *0.9^{x/250}   notes：x is the current generation number
#tensorflow 有个函数为 staircase
learning_rate = 0.1
lr_decay = 0.9
num_gens_to_wait = 250.

In [4]:
image_vec_length = image_height * image_width * num_channels
record_length = 1 + image_vec_length  # label 1  input 32*32*3

In [5]:
data_dir='./CNNdata/cifar-10-binary/cifar-10-batches-bin'   #data所在目录底下

def read_cifar_files(filename_queue, distort_images = True):
    reader = tf.FixedLengthRecordReader(record_bytes=record_length) #定长读取
    key, record_string = reader.read(filename_queue)
    record_bytes = tf.decode_raw(record_string, tf.uint8)
    image_label = tf.cast(tf.slice(record_bytes, [0], [1]), tf.int32) #slice 切割 label
    image_extracted = tf.reshape(tf.slice(record_bytes, [1], [image_vec_length]), [num_channels, image_height, image_width])
    image_uint8image = tf.transpose(image_extracted, [1, 2, 0])
    reshaped_image = tf.cast(image_uint8image, tf.float32)
    final_image = tf.image.resize_image_with_crop_or_pad(reshaped_image, crop_width, crop_height)
    if distort_images:
        final_image = tf.image.random_flip_left_right(final_image)
        final_image = tf.image.random_brightness(final_image,max_delta=63)
        final_image = tf.image.random_contrast(final_image,lower=0.2, upper=1.8)
        final_image = tf.image.per_image_whitening(final_image)
    return(final_image, image_label)


In [6]:
def input_pipeline(batch_size, train_logical=True):
     if train_logical:
         files = [os.path.join(data_dir,'data_batch_{}.bin'.format(i)) for i in range(1,6)]
     else:
         files = [os.path.join(data_dir, 'test_batch.bin')]
     filename_queue = tf.train.string_input_producer(files) #输出字符串到一个管道队列
     image, label = read_cifar_files(filename_queue)        #读取一张图片
     min_after_dequeue = 1000
     capacity = min_after_dequeue + 3 * batch_size
     example_batch, label_batch = tf.train.shuffle_batch([image, label], batch_size, capacity, min_after_dequeue)
     return(example_batch, label_batch)